# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [ ]:
data_link = 'https://github.com/fcgomezr/Udacity-Project-3/blob/main/data%20sets/bank-full.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from model_training import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data.head()

In [ ]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

In [ ]:
train_data.to_csv("traindata.csv")

In [ ]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="y",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

In [ ]:
print(best_automl)

In [ ]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

In [ ]:
joblib.dump(best_fit_model,'./outputs/bestmodel.joblib')
best_class.register_model(model_name='best_fit_automl_model.pkl',model_path='/outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Web Service Logs
print(service.get_logs())

In [ ]:
# Delete the service
service.delete()